In [1]:
import tensorflow as tf
import time

In [2]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        # open file
        time.sleep(0.03)
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield(sample_idx,)

    def __new__(cls,num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature=tf.TensorSpec(shape=(1,),dtype=tf.int64),
            args=(num_samples,)
        )

In [3]:
def benchmark(dataset,num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            time.sleep(0.01)

In [4]:
%%timeit
benchmark(FileDataset())

834 ms ± 116 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

763 ms ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
dataset=tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [10]:
dataset=dataset.map(lambda x:x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [11]:
datset=dataset.cache()
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [13]:
def mapped_function(s):
    tf.py_function(lambda:time.sleep(0.03),[],())
    return s

In [14]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function),5)

1.71 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function).cache(),5)

676 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
